In [1]:
from os.path import join as os_path_join

from scipy.io import loadmat, savemat
from numpy import zeros as np_zeros, ones as np_ones, zeros_like as np_zeros_like
from numpy import vectorize as np_vectorize
from numpy.linalg import norm
from numpy import spacing as np_spacing
from numpy import log10 as np_log10
from numpy import clip as np_clip, multiply as np_multiply

from lib.better_envelope import better_envelope
# from lib.stft import stft
# from lib.non_iter_ls_inv_stft import non_iter_ls_inv_stft
target_dirname = '/Users/zhanwenchen/Documents/projects/beam_nn/DNNs/test/scan_batteries/target_phantom_anechoic_cyst_2p5mm/target_1/'

# # Test variables
# num_rows, num_elements, num_beams = 1624, 65, 128
# len_each_section = 16;
# frac_overlap = 0.9;
# padding = 16;
# # winInfo = {@rectwin};
# CHANDAT_FNAME = '/Users/zhanwenchen/Documents/projects/beam_nn/DNNs/test/scan_batteries/target_phantom_anechoic_cyst_2p5mm/target_1/chandat.mat'
# chandat = loadmat(CHANDAT_FNAME)['chandat']

# NEW_STFT_FNAME = '/Users/zhanwenchen/Documents/projects/beam_nn/DNNs/test/scan_batteries/target_phantom_anechoic_cyst_2p5mm/target_1/new_stft.mat'
# new_stft_f = loadmat(NEW_STFT_FNAME)
# new_stft_real = new_stft_f['new_stft_real']
# new_stft_imag = new_stft_f['new_stft_imag']


In [2]:
from lib.r5_dnn_image import r5_dnn_image

In [3]:
r5_dnn_image(target_dirname)

In [71]:
# r5 test vars
from scipy.signal import butter, filtfilt, lfilter


CHANDAT_FNAME = 'chandat.mat'
CHANDAT_DNN_FNAME = 'chandat_dnn.mat'
CHANDAT_IMAGE_SAVE_FNAME = 'chandat_image.mat'

chandat_obj = loadmat(os_path_join(target_dirname, CHANDAT_FNAME))
f0 = chandat_obj['f0']
chandat_dnn_obj = loadmat(os_path_join(target_dirname, CHANDAT_DNN_FNAME))
chandat_dnn = chandat_dnn_obj['chandat_dnn']
beam_position_x = chandat_dnn_obj['beam_position_x']
if f0.ndim and f0.ndim == 2:
    f0 = f0[0, 0]
    

In [3]:
# design a bandpass filter
n = 4
Wn = [1e6, 9e6]/(4*f0/2)
b, a = butter(n/2, Wn, btype='bandpass') # Results are correct

In [4]:
chandat_dnn = chandat_dnn.astype(float, copy=False)

In [5]:
rf_data = chandat_dnn.sum(axis=1)

In [6]:
rf_data_filt = filtfilt(b, a, rf_data, axis=0, padtype='odd', padlen=3*(max(len(b),len(a))-1)) # Correct


In [7]:
# from scipy.signal import remez

# cutoff_low = 0.2
# cutoff_high = 0.8
# # fs = 20.832
# num_taps = 10

# # This is equivalent to B=firpm(10,[.2 .8],[1 1],'Hilbert');
# coefficients = remez(num_taps+1, [cutoff_low/2, cutoff_high/2], [1], type='hilbert')





In [8]:
# from numpy import sqrt as np_sqrt

# env = np_zeros_like(rf_data_filt)

# test_input = rf_data_filt[:, 0]
# Q = filtfilt(coefficients, 1, test_input)
# print('norm(Q)={}'.format(norm(Q)))
# envelope = np_sqrt(test_input ** 2 + Q ** 2)
# print('norm(envelope)={}'.format(norm(envelope)))
# b, a = butter(5, 0.25, btype='low')
# print('norm(b)={}; norm(a)={}'.format(norm(b), norm(a)))

# envelope_filtered = filtfilt(b, a, envelope, axis=0, padtype='odd', padlen=3*(max(len(b),len(a))-1))
# print('norm(envelope_filtered)={}'.format(norm(envelope_filtered)))

In [9]:
# envelope_filtered.clip(min=0, out=envelope_filtered)

# test_output = better_envelope(test_input)

# print('norm(test_input) = {}; test_input[0]={}'.format(norm(test_input), test_input[0]))
# print('norm(test_output) = {}; test_output[0]={}'.format(norm(test_output), test_output[0]))

# for k in range(env.shape[1]):
#     env[:, k] = better_envelope(rf_data_filt[:, k])
    
# print('norm(env) = {}; env[0,0]={}'.format(norm(env), env[0,0]))

In [10]:
from numpy import apply_along_axis

from lib.better_envelope import better_envelope

env = apply_along_axis(better_envelope, 0, rf_data_filt)

In [25]:
# Correct
env = env / env.max()
np_clip(env, np_spacing(1), None, out=env)
env_dB = np_zeros_like(env)
np_log10(env, out=env_dB);
np_multiply(env_dB, 20, out=env_dB);


In [29]:
from scipy.interpolate import pchip

# Upscale lateral sampling
up_scale = 2
env_up = []


In [31]:
env.shape

(1624, 128)

In [ ]:
pchip(range(1, 128))

In [49]:
x = range(1, 129)
y = env[0, :]

In [47]:
from numpy import arange as np_arange

new_x = np_arange(1, env.shape(1)+1/up_scale, 1/up_scale)

In [57]:
new_y = pchip(x, y)(new_x) # correct

In [56]:
norm(new_y)

0.06135190298290696

In [58]:
def curried_pchip(y):
    return pchip(x, y)(new_x)

In [59]:
env_up = apply_along_axis(curried_pchip, 1, env)

In [60]:
norm(env_up)

47.36790818710541

In [64]:
np_clip(env_up, np_spacing(1), None, out=env_up);
env_up_dB = np_zeros_like(env_up)
np_log10(env_up, out=env_up_dB);
np_multiply(env_up_dB, 20, out=env_up_dB);


In [65]:
norm(env_up_dB)

22715.68410845773

In [75]:
from numpy import linspace as np_linspace

beam_position_x_up = np_linspace(beam_position_x.min(), beam_position_x.max(), env_up.shape[1])


In [81]:
beam_position_x_up[-1]

0.018923000000000002

In [ ]:
# chandat_image_path = os_path_join(target_dirname, CHANDAT_IMAGE_SAVE_FNAME)
# savemat(chandat_image_path, {
#     'rf_data': rf_data,
#     'rf_data_filt': rf_data_filt,
#     'env': env,
#     'env_dB': env_dB,
#     'env_up': env_up,
#     'env_up_dB': env_up_dB,
#     'beam_position_x_up': beam_position_x_up,
#     'depth': depth,
# })

In [ ]:
# save('dnn_image.mat', 'rf_data', 'rf_data_filt', 'env', 'env_dB', 'envUp', ... 
#         'envUp_dB', 'beam_position_x_up', 'depth')

In [ ]:
#  list(range(1,129)), env[0, :]
# envUp(kk,:) = interp1((1:size(env,2)), env(kk,:), (1:(1/upScale):size(env,2)), 'pchip');

In [ ]:
# % upscale lateral sampling
# upScale = 2;
# envUp = [];
# for kk = 1:size(env,1)
#    envUp(kk,:) = interp1((1:size(env,2)), env(kk,:), (1:(1/upScale):size(env,2)), 'pchip');
# end
# envUp(envUp <= 0) = eps;
# envUp_dB = 20*log10(envUp);
# beam_position_x_up = linspace(min(beam_position_x), max(beam_position_x), size(envUp, 2));

# % save dnn workspace
# save('dnn_image.mat', 'rf_data', 'rf_data_filt', 'env', 'env_dB', 'envUp', ... 
#         'envUp_dB', 'beam_position_x_up', 'depth')
